In [ ]:
import numpy as np
import os
import rasterio as rio
import xarray as xr
from datetime import datetime
from rasterio.warp import transform

from carbonplan_data import cat

## Example grid file

<details>
<pre>
netcdf domain.lnd.wr50a_ar9v4.100920 {
dimensions:
	n = 56375 ;
	ni = 275 ;
	nj = 205 ;
	nv = 4 ;
variables:
	double xc(nj, ni) ;
		xc:long_name = "longitude of grid cell center" ;
		xc:units = "degrees_east" ;
		xc:bounds = "xv" ;
	double yc(nj, ni) ;
		yc:long_name = "latitude of grid cell center" ;
		yc:units = "degrees_north" ;
		yc:bounds = "yv" ;
	double xv(nj, ni, nv) ;
		xv:long_name = "longitude of grid cell verticies" ;
		xv:units = "degrees_east" ;
	double yv(nj, ni, nv) ;
		yv:long_name = "latitude of grid cell verticies" ;
		yv:units = "degrees_north" ;
	int mask(nj, ni) ;
		mask:long_name = "domain mask" ;
		mask:note = "unitless" ;
		mask:coordinates = "xc yc" ;
		mask:comment = "0 value indicates cell is not active" ;
	double area(nj, ni) ;
		area:long_name = "area of grid cell in radians squared" ;
		area:coordinates = "xc yc" ;
		area:units = "radian2" ;
	double frac(nj, ni) ;
		frac:long_name = "fraction of grid cell that is active" ;
		frac:coordinates = "xc yc" ;
		frac:note = "unitless" ;
		frac:filter1 = "error if frac> 1.0+eps or frac < 0.0-eps; eps = 0.1000000E-10" ;
		frac:filter2 = "limit frac to [fminval,fmaxval]; fminval= 0.1000000E-02 fmaxval=  1.000000" ;

// global attributes: :title = "CCSM domain data:" ; :Conventions = "CF-1.0" ;
:source_code = "SVN $Id: gen_domain.F90 6673 2007-09-28 22:11:15Z kauff $" ;
:SVN_url = "
$URL: https://svn-ccsm-models.cgd.ucar.edu/tools/mapping/gen_domain/trunk_tags/gen_domain_071001/gen_domain.F90 $"
; :history = "created by tcraig, 2010-09-21 13:41:55" ; :source =
"/fis/cgd/cseg/csm/inputdata/cpl/cpl6/map_ar9v4_to_wr50a_aave_da_100920.nc" ;
:map_domain_a = "RACM ar9v4 grid" ; :map_domain_b = "WRF RACM grid 275x205" ;
:map_grid_file_ocn = "unknown" ; :map_grid_file_atm = "unknown" ; :output_file1
= "domain.ocn.ar9v4.100920.nc" ; :output_file2 =
"domain.lnd.wr50a_ar9v4.100920.nc" ; :user_comment = "Standard CCSM3.1 domain
specification file" ; </pre>

</details>


In [ ]:
region = "ak"

nlcd_url = f"https://storage.googleapis.com/carbonplan-data/processed/nlcd/{region}/4000m/2016.tif"

riods = rio.open(nlcd_url)

In [ ]:
def extract_grid_coords(tiff_url):

    out_lon = {}
    out_lat = {}

    for offset in ["center", "ul", "ll", "ur", "lr"]:
        tempx = np.empty(shape=riods.shape, dtype=float)
        tempy = np.empty(shape=riods.shape, dtype=float)

        for (row, col), _ in np.ndenumerate(tempx):
            x, y = riods.xy(row, col, offset=offset)
            tempx[(row, col)] = x
            tempy[(row, col)] = y

        lon, lat = transform(
            riods.crs, {"init": "EPSG:4326"}, tempx.flatten(), tempy.flatten()
        )

        out_lon[offset] = np.asarray(lon).reshape(riods.shape)
        out_lat[offset] = np.asarray(lat).reshape(riods.shape)

    return out_lon, out_lat

In [ ]:
nlcd_coords = extract_grid_coords(nlcd_url)

In [ ]:
crs = riods.crs
crs_dict = crs.to_dict()
crs_dict

In [ ]:
crs.to_string()

In [ ]:
user = os.environ["JUPYTERHUB_USER"]
global_attrs = {
    "title": f"Albers 4k {region.upper()} Domain",
    "Conventions": "CF-1.8",
    "history": f"created by {user}, {datetime.now()}",
}
dxdy = 4000.0  # m

if crs_dict["proj"] != "aea":
    raise AssertionError("projection needs to be Albers_Conical_Equal_Area")


attrs = {
    "lon": {
        "long_name": "longitude of grid cell center",
        "units": "degrees_east",
        "bounds": "lon_b",
    },
    "lat": {
        "long_name": "latitude of grid cell center",
        "units": "degrees_north",
        "bounds": "lat_b",
    },
    "lon_b": {
        "long_name": "longitude of grid cell verticies",
        "units": "degrees_east",
    },
    "lat_b": {
        "long_name": "latitude of grid cell verticies",
        "units": "degrees_east",
    },
    "mask": {
        "long_name": "domain mask",
        "note": "unitless",
        "coordinates": "xc yc",
        "comment": "0 value indicates cell is not active",
    },
    "area": {
        "long_name": "area of grid cell in meters squared",
        "coordinates": "xc yc",
        "units": "m2",
    },
    "x": {
        "long_name": "x coordinate of projection",
        "standard_name": "projection_x_coordinate",
        "units": "m",
    },
    "y": {
        "long_name": "x coordinate of projection",
        "standard_name": "projection_y_coordinate",
        "units": "m",
    },
    "crs": {
        "grid_mapping_name": "albers_conical_equal_area",
        "standard_parallel": [crs_dict["lat_1"], crs_dict["lat_2"]],
        "longitude_of_central_meridian": crs_dict["lon_0"],
        "latitude_of_projection_origin": crs_dict["lat_0"],
        "false_easting": crs_dict["x_0"],
        "false_northing": crs_dict["y_0"],
        "crs_wkt": crs.to_wkt(),
    },
}

grid = xr.Dataset(attrs=global_attrs)
grid

In [ ]:
lons, lats = nlcd_coords
grid["lon"] = xr.DataArray(
    lons["center"], name="lon", dims=("y", "x"), attrs=attrs["lon"]
)
grid["lat"] = xr.DataArray(
    lats["center"], name="lat", dims=("y", "x"), attrs=attrs["lat"]
)

grid

In [ ]:
# add grid cell boundaries
shape = grid.lon.shape
shape = tuple(s + 1 for s in shape)

lon_b = np.full(shape, np.nan)
lon_b[:-1, :-1] = lons["ul"]
lon_b[:-1, -1] = lons["ur"][:, -1]
lon_b[-1, :-1] = lons["ll"][-1, :]
lon_b[-1, -1] = lons["lr"][-1, -1]

lat_b = np.full(shape, np.nan)
lat_b[:-1, :-1] = lats["ul"]
lat_b[:-1, -1] = lats["ur"][:, -1]
lat_b[-1, :-1] = lats["ll"][-1, :]
lat_b[-1, -1] = lats["lr"][-1, -1]

grid["lon_b"] = xr.DataArray(
    lon_b, name="lon_b", dims=("y_b", "x_b"), attrs=attrs["lon_b"]
)
grid["lat_b"] = xr.DataArray(
    lat_b, name="lat_b", dims=("y_b", "x_b"), attrs=attrs["lat_b"]
)

# uncomment to use (i, j, 4) notation
# grid['xv'] = xr.DataArray(np.stack([lons[k] for k in ['ul', 'ur', 'lr', 'll']], axis=2), name='xv', dims=('nj', 'ni', 'nv'), attrs=attrs['xv'])
# grid['yv'] = xr.DataArray(np.stack([lats[k] for k in ['ul', 'ur', 'lr', 'll']], axis=2), name='yv', dims=('nj', 'ni', 'nv'), attrs=attrs['yv'])
grid

In [ ]:
da = cat.nlcd.raster(region=region).read()
grid["mask"] = xr.where(da.squeeze(drop=True), 1, 0).astype(np.int)
grid["area"] = xr.zeros_like(grid["mask"]) + (dxdy ** 2)
grid["x"] = da.coords["x"]
grid["y"] = da.coords["y"]
grid["crs"] = xr.DataArray(1, name="crs")

for var in ["mask", "area", "x", "y", "crs"]:
    grid[var].attrs = attrs[var]

grid

In [ ]:
grid.info()

In [ ]:
import gcsfs

fs = gcsfs.GCSFileSystem(
    project="carbonplan", token="cloud", requester_pays=True
)
mapper = fs.get_mapper(
    f"carbonplan-data/processed/grids/{region}/4000m/domain.zarr"
)
grid.to_zarr(mapper, mode="w", consolidated=True)

### todo:

- down-sample 30m grid mask to create `frac` variable
